<a href="https://colab.research.google.com/github/JuanDiaz77/Proyecto-colab/blob/main/Proyecto_Capstone.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:

!pip install scikit-learn joblib pandas fastapi uvicorn[standard] nest-asyncio pyngrok streamlit great_expectations -q


In [ ]:
import random
import pandas as pd
from pathlib import Path

output_dir = Path("/content/proyecto_capstone")
output_dir.mkdir(exist_ok=True)

# simulacion de PQRS
classes = ["PQR", "felicitacion", "queja", "solicitud", "denuncia"]
templates = {
    "PQR": ["Quisiera información sobre el estado de mi trámite {}",
            "Solicito copia del documento {}",
            "¿Cuál es el procedimiento para {}?"],
    "felicitacion": ["Gracias por el excelente servicio en la oficina de {}",
                     "Quiero felicitar al equipo de {} por su atención",
                     "Muy agradecido por la ayuda recibida en {}"],
    "queja": ["Presento una queja por la demora en {}",
              "Insatisfecho con el servicio recibido en {}",
              "No recibí respuesta sobre {}"],
    "solicitud": ["Solicito la reparación de la vía en {}",
                  "Pido apoyo para el servicio de {}",
                  "Necesito agendar una cita para {}"],
    "denuncia": ["Deseo denunciar un robo en {}",
                "Reporto actividades sospechosas cerca de {}",
                "Denuncio maltrato en {}"]
}
places = ["la alcaldía", "la oficina de tránsito", "el centro de atención", "la calle 45", "la sede principal", "la estación"]

rows = []
for i in range(600):
    cls = random.choice(classes)
    template = random.choice(templates[cls])
    place = random.choice(places)
    text = template.format(place)
    if random.random() < 0.2:
        text += " por favor"
    if random.random() < 0.15:
        text = text.replace("Solicito", "Quisiera solicitar")
    rows.append({"texto": text, "tipo": cls})

df = pd.DataFrame(rows)
df.to_csv(output_dir / "dataset_solicitudes.csv", index=False, encoding="utf-8-sig")
df.head()


In [ ]:
# Este código entrena un modelo de clasificación de texto (NLP) usando un pipeline con TF-IDF y Regresión Logística, y luego guarda el modelo entrenado en un archivo (model_pipeline.joblib).

# Importa las librerias necesarias
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
import joblib
import json

#Separacion de los datos
X = df["texto"].values
y = df["tipo"].values

#separacion de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

#Definicon del Pipilen
pipeline = Pipeline([
    ("tfidf", TfidfVectorizer(ngram_range=(1,2), min_df=2)),
    ("clf", LogisticRegression(max_iter=1000))
])

pipeline.fit(X_train, y_train)

# Guarda modelo en el workspace de Colab
model_path = output_dir / "model_pipeline.joblib"
joblib.dump(pipeline, model_path)


In [ ]:
#evalúa el modelo entrenado y guarda los resultados de la evaluación en archivos .csv.

#Importa las metricas

from sklearn.metrics import confusion_matrix, classification_report
import pandas as pd

# Genera las prediciones
y_pred = pipeline.predict(X_test)

#Define las clases
classes = ["PQR","felicitacion","queja","solicitud","denuncia"]

#Calcula la matriz de confusion: Determina cuantas veces el modelo se equivoco y acerto en cada clase.
cm = confusion_matrix(y_test, y_pred, labels=classes)

#Calcula el reporte de clasificacion - se guarda como diccionario para facilitar la convercion a tabla
report = classification_report(y_test, y_pred, target_names=classes, output_dict=True)

#guarda el resultado
pd.DataFrame(cm, index=classes, columns=classes).to_csv(output_dir / "confusion_matrix.csv", encoding="utf-8-sig")
pd.DataFrame(report).to_csv(output_dir / "classification_report.csv", encoding="utf-8-sig")

#Guarda las predicciones para retroalimentar el modelo
preds_df = pd.DataFrame({"texto": X_test, "true": y_test, "pred": y_pred})
preds_df.to_csv(output_dir / "predicciones_test.csv", index=False, encoding="utf-8-sig")


In [ ]:
#Entrenamiento y guardado del Script

#Importacion de librerias y funciones
from fastapi import FastAPI
from pydantic import BaseModel
import joblib

# ruta:Carga desde el disco el pipeline entrenado (TF-IDF + Regresión Logística)
model = joblib.load("/content/proyecto_capstone/model_pipeline.joblib")

#Crea la aplicación FastAPI y le asigna un título
app = FastAPI(title="Clasificador de solicitudes ciudadanas")

# Define el formato del JSON que se debe enviar al endpoint /predict.
class InputText(BaseModel):
    texto: str

#Define un endpoint tipo POST llamado /predict.
@app.post("/predict")
# recibe un objeto del tipo InputText
def predict(item: InputText):
  #Extrae el texto enviado del modelo recibido.
    texto = item.texto
    #Llama al modelo para predecir la clase del texto y toma la predicion.
    pred = model.predict([texto])[0]
    #Toma la probabilidad mas alta
    proba = float(model.predict_proba([texto]).max())
    #Devuelve la respuesta en formato JSON
    return {"prediccion": pred, "probabilidad": proba}

#Define un enpoint de tipo GET
@app.get("/")
def root():
  #Funcion de retorno de estado
    return {"status": "ok", "mensaje": "API funcionando correctamente"}


In [ ]:
#Libreria para exponer la API mediante URL publica
from pyngrok import ngrok
import nest_asyncio
import uvicorn
import threading
import time
import os

#Habilita la reutilización del mismo bucle asincrónico, necesario para que FastAPI funcione correctamente dentro de Google Colab.
nest_asyncio.apply()

# rectifica la carpeta del proyecto para uvicorn
os.chdir("/content/proyecto_capstone")

#Define el host y el puerto para ejecutar la API
def run_uvicorn():
    uvicorn.run("app:app", host="0.0.0.0", port=8000)

#hilo de ejecución paralelo que correrá Uvicorn.
thread = threading.Thread(target=run_uvicorn, daemon=True)
#Inicia el servidor de FASTAPI
thread.start()
time.sleep(2)

#Crea el tunel publico y URL
public_url = ngrok.connect(8000).public_url
print(" API en ejecución en:", public_url)


In [ ]:
# Desactiva el puerto
!fuser -k 8000/tcp


In [ ]:
# Token de logueo ngrok
!ngrok authtoken 34nWVluqfWCUt0chClyAvO3RNMc_6DtsZ1KjCxj3YhTCeLGiV


In [ ]:
#Cambia el directorio de la carpeta
import os
os.chdir("/content/proyecto_capstone")
!ls


In [ ]:
#guarda el contenido del bloque para la compatibilidad en colab
%%writefile app.py
from fastapi import FastAPI
from pydantic import BaseModel
import joblib

app = FastAPI(title="Clasificador de solicitudes ciudadanas")

# Cargar el modelo de machine learning
model = joblib.load("model_pipeline.joblib")

class InputText(BaseModel):
    texto: str

#Crea la ruta POST para recibir el texto y clasificar
@app.post("/predict")
def predict(item: InputText):
    texto = item.texto
    pred = model.predict([texto])[0]
    proba = float(model.predict_proba([texto]).max())
    return {"prediccion": pred, "probabilidad": proba}

@app.get("/")
def root():
    return {"status": "ok", "mensaje": "API funcionando correctamente"}


In [ ]:
%%writefile /content/proyecto_capstone/validate.py
import pandas as pd
import great_expectations as gx

# Cargar dataset
df = pd.read_csv("/content/proyecto_capstone/dataset_solicitudes.csv")

print(" Iniciando validación de calidad de datos...\n")

# Expectativas básicas
expectations = {
    "no valores nulos en texto": df["texto"].notnull().all(),
    "no valores nulos en tipo": df["tipo"].notnull().all(),
    "longitud mínima del texto > 5": (df["texto"].str.len() > 5).all(),
    "clases válidas": set(df["tipo"].unique()).issubset({"PQR","felicitacion","queja","solicitud","denuncia"})
}

for check, result in expectations.items():
    print(f"- {check}: {' OK' if result else ' Falla'}")

# Guardar resumen como CSV
summary = pd.DataFrame(list(expectations.items()), columns=["check", "status"])
summary.to_csv("/content/proyecto_capstone/validacion_resumen.csv", index=False)

print("\n📄 Resultados guardados en validacion_resumen.csv")
print(" Validación terminada.")


In [ ]:
#Instala la libreria para validar la calidad de los datos y la instala en colab
!pip install great_expectations
!python validate.py



In [ ]:
!python validate.py


In [ ]:
texto = """# Informe Breve del Proyecto Capstone

## Día 1: Entrenamiento del modelo
Se creó un dataset de solicitudes ciudadanas simuladas (600 filas).
Se entrenó un clasificador con TF-IDF + LogisticRegression y se evaluó con matriz de confusión y reporte de clasificación.

## Día 2: Despliegue simulado
Se desarrolló una API con FastAPI para clasificar texto en categorías de tipo de solicitud.
Se empaquetó el servicio con Dockerfile, y se documentó la configuración para despliegue simulado en Google Cloud Run.

## Día 3: Validación y visualización
Se validó el dataset con Great Expectations (sin nulos, columnas correctas, clases válidas).
Los resultados fueron 100 % exitosos.
Se generaron archivos de log y CSV con los resultados.

---
Autor: Juan Pablo Mora Diaz
Fecha: 30/10/2025
"""
from pathlib import Path
Path("/content/proyecto_capstone/informe_breve.md").write_text(texto, encoding="utf-8")
print("Informe creado ")


In [ ]:
import os, zipfile, textwrap, subprocess, sys
from pathlib import Path

proj = Path("/content/proyecto_capstone")
if not proj.exists():
    raise SystemExit(f"No encontré la carpeta {proj}. Asegúrate de que los archivos estén aquí.")

md_file = proj / "informe_breve.md"
pdf_file = proj / "informe_breve.pdf"
if md_file.exists():
    try:
        try:
            import reportlab
        except Exception:
            subprocess.check_call([sys.executable, "-m", "pip", "install", "reportlab"], stdout=subprocess.DEVNULL)
        from reportlab.lib.pagesizes import letter
        from reportlab.pdfgen import canvas
        text = md_file.read_text(encoding="utf-8")
        c = canvas.Canvas(str(pdf_file), pagesize=letter)
        width, height = letter
        margin = 40
        y = height - margin
        for paragraph in text.split("\n\n"):
            for line in textwrap.wrap(paragraph, width=95):
                if y < margin + 20:
                    c.showPage()
                    y = height - margin
                c.drawString(margin, y, line)
                y -= 12
            y -= 8
        c.save()
        print("PDF generado:", pdf_file)
    except Exception as e:
        print("No se pudo generar PDF automáticamente:", e)
else:
    print("No se encontró informe_breve.md — saltando generación de PDF.")

files = [
    "dataset_solicitudes.csv",
    "notebook_entrenamiento.ipynb",
    "model_pipeline.joblib",
    "confusion_matrix.csv",
    "classification_report.csv",
    "predicciones_test.csv",
    "entrenamiento.log",
    "app.py",
    "Dockerfile",
    "requirements.txt",
    "streamlit_app.py",
    "validate.py",
    "validacion_resumen.csv",
    "validacion_resultados.log",
    "informe_breve.md",
    "informe_breve.docx",
    "informe_breve.pdf"
]

files = [f for f in files if (proj / f).exists()]

zip_path = Path("/content/Proyecto_Capstone_entregable.zip")
with zipfile.ZipFile(zip_path, "w", compression=zipfile.ZIP_DEFLATED) as zf:
    for fname in files:
        zf.write(proj / fname, arcname=fname)

print("ZIP creado en:", zip_path)
print("Archivos incluidos:")
for f in files:
    print(" -", f)


In [ ]:
from google.colab import files
files.download("/content/Proyecto_Capstone_entregable.zip")


In [ ]:
#Lista los archivos actuales
!ls /content/proyecto_capstone
